In [ ]:
from pydantic import BaseModel
from typing import List, Dict, Any
from ogpu.client import publish_task, TaskInfo, TaskInput, ChainConfig, ChainId
from web3 import Web3

ChainConfig.set_chain(chain_id=ChainId.OGPU_MAINNET)

In [ ]:
class Inference(BaseModel):
    input: str | dict
    args : dict  = {}

In [ ]:
class DetectionOutput(BaseModel):
    output: List[Dict[str, Any]]
    
    class Config:
        # This allows custom __str__ and __repr__ to work with Pydantic
        arbitrary_types_allowed = True
    
    def display_output(self):
        """Display detection results in a nice format"""
        print("🔍 Object Detection Results")
        print("=" * 50)
        
        print(f"Detected {len(self.output)} objects:")
        # Sort by confidence score (highest first)
        sorted_output = sorted(self.output, key=lambda x: x['score'], reverse=True)
        
        for i, result in enumerate(sorted_output, 1):
            confidence_percent = result['score'] * 100
            
            # Color coding based on confidence
            if confidence_percent >= 90:
                confidence_emoji = "🟢"
            elif confidence_percent >= 70:
                confidence_emoji = "🟡"
            else:
                confidence_emoji = "🔴"
                
            print(f"\n{i}. {confidence_emoji} {result['label'].upper()}")
            print(f"   Confidence: {confidence_percent:.1f}%")
            box = result['box']
            print(f"   Bounding Box: ({box['xmin']}, {box['ymin']}) → ({box['xmax']}, {box['ymax']})")

    def __str__(self):
        """Custom string representation for print()"""
        return self._format_results()
    
    def __repr__(self):
        """Custom representation for Jupyter notebook display"""
        return self._format_results()
    
    def _format_results(self):
        """Helper method to format results consistently"""
        result_lines = []
        result_lines.append("🔍 Object Detection Results")
        result_lines.append("=" * 50)
        
        result_lines.append(f"Detected {len(self.output)} objects:")
        # Sort by confidence score (highest first)
        sorted_output = sorted(self.output, key=lambda x: x['score'], reverse=True)
        
        for i, result in enumerate(sorted_output, 1):
            confidence_percent = result['score'] * 100
            
            # Color coding based on confidence
            if confidence_percent >= 90:
                confidence_emoji = "🟢"
            elif confidence_percent >= 70:
                confidence_emoji = "🟡"
            else:
                confidence_emoji = "🔴"
                
            result_lines.append(f"\n{i}. {confidence_emoji} {result['label'].upper()}")
            result_lines.append(f"   Confidence: {confidence_percent:.1f}%")
            box = result['box']
            result_lines.append(f"   Bounding Box: ({box['xmin']}, {box['ymin']}) → ({box['xmax']}, {box['ymax']})")
        
        return "\n".join(result_lines)
    
    def _repr_pretty_(self, p, cycle):
        """For IPython/Jupyter pretty printing"""
        p.text(self._format_results())

In [ ]:
inference_data = Inference(
    input= "https://lh7-us.googleusercontent.com/GNVsVvVmCDLtFtqburTWaTQOjXn3N0wLBKXt6BhXO2GG037S9o7xZ7HyPjFcJoWgds1PFKvbHgCCEWTqiB-MR0VFIo7UC7jVEX6t10LD6zPg61YoEAR-NGbA2RJj7dVdYwMH2nsOJY648uXmMlpNiZw",
    args= {
        "threshold": 0.5  # minimum confidence score
    }
)

task_input = TaskInput(
    function_name="inference",
    data = inference_data
)

In [ ]:
task_info = TaskInfo(
    source="0xaddy",
    config=task_input,
    expiryTime=86400,  # 24 hour in seconds
    payment=Web3.to_wei(0.001, "ether"),
)

task_address = publish_task(task_info)
print(f"Task published at address: {task_address}")

In [ ]:
from ogpu.client import get_confirmed_response

response = get_confirmed_response(task_address)

# Recreate with updated class
det_res = DetectionOutput(output=response.data["result"]['output'])
det_res